In [ ]:
import os
import random
import copy
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import layers, models, optimizers
from keras.preprocessing import image_dataset_from_directory
from keras.applications import (
    ResNet50,
    EfficientNetB0,
)
from keras.applications.resnet50 import preprocess_input

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, top_k_accuracy_score
import seaborn as sns
import wandb

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nunenuh/pytorch-challange-flower-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'pytorch-challange-flower-dataset' dataset.
Path to dataset files: /kaggle/input/pytorch-challange-flower-dataset


In [ ]:
import os

print(path)
print(os.listdir(path))


/kaggle/input/pytorch-challange-flower-dataset
['sample_submission.csv', 'dataset', 'README.md', 'cat_to_name.json']


In [ ]:
import tensorflow as tf
import os

IMG_SIZE = 224
BATCH_SIZE = 16   # Kaggle T4 an toàn hơn 32

BASE_PATH = "/kaggle/input/pytorch-challange-flower-dataset/dataset"

train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(BASE_PATH, "train"),
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="int"
)


val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(BASE_PATH, "valid"),
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="int"
)


Found 6552 files belonging to 102 classes.
Found 818 files belonging to 102 classes.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def normalize(x, y):
    return tf.cast(x, tf.float32) / 255.0, y

train_ds = train_ds.map(normalize).shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.map(normalize).prefetch(AUTOTUNE)


In [ ]:
path = kagglehub.model_download(
    "keras/vit/keras/vit_base_patch16_224_imagenet"
)
print(path)


/kaggle/input/vit/keras/vit_base_patch16_224_imagenet/3


In [ ]:
import keras
import keras_hub
from keras import layers
backbone = keras_hub.models.ViTBackbone.from_preset(
    "vit_base_patch16_224_imagenet"
)
inputs = keras.Input(shape=(224, 224, 3))
x = backbone(inputs)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(102, activation="softmax")(x)

model = keras.Model(inputs, outputs)

def calculate_all_metrics(y_true, y_pred, y_pred_probs):
    """Calculate all required metrics"""
    acc = accuracy_score(y_true, y_pred)
    top5_acc = top_k_accuracy_score(y_true, y_pred_probs, k=5)
    f1_macro = f1_score(y_true, y_pred, average="macro", zero_division=0)
    f1_weighted = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    return acc, top5_acc, f1_macro, f1_weighted

def collect_predictions(model, dataset):
    """Collect predictions from dataset"""
    y_true_all, y_pred_all, y_pred_probs_all = [], [], []
    for x_batch, y_batch in dataset:
        batch_probs = model.predict(x_batch, verbose=0)
        batch_preds = np.argmax(batch_probs, axis=1)
        y_true_all.extend(y_batch.numpy())
        y_pred_all.extend(batch_preds)
        y_pred_probs_all.extend(batch_probs)
    return np.array(y_true_all), np.array(y_pred_all), np.array(y_pred_probs_all)

In [ ]:
# ====== ONE-CELL ViT FINETUNING (FREEZE → UNFREEZE) ======

import keras
import keras_hub
from keras import layers

NUM_CLASSES = 102
IMG_SIZE = 224


wandb.init(
    project="flower-classification-vit",
    config={
        "learning_rate": 1e-3,
        "architecture": "ViT-Base-Patch16-224",
        "dataset": "Flowers102",
        "epochs": 10,
        "batch_size": BATCH_SIZE,
        "img_size": IMG_SIZE,
        "num_classes": NUM_CLASSES
    }
)

# 1. Load pretrained ViT backbone (ImageNet)
backbone = keras_hub.models.ViTBackbone.from_preset(
    "vit_base_patch16_224_imagenet"
)

# 2. PHASE 1: Freeze backbone
backbone.trainable = False

# 3. Build model
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = backbone(inputs)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)
model = keras.Model(inputs, outputs)

wandb.config.update({"model_summary": str(model.summary())})

# 4. Compile & train (head only)
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
# Phase 1 Training with wandb logging
print("Phase 1: Training head only (backbone frozen)")
for epoch in range(5):
    print(f"\nEpoch {epoch+1}/5")

    # Training
    train_loss = 0
    train_acc = 0
    train_batches = 0

    for batch_idx, (x_batch, y_batch) in enumerate(train_ds):
        # Train on batch
        loss, acc = model.train_on_batch(x_batch, y_batch)
        train_loss += loss
        train_acc += acc
        train_batches += 1

        # Log batch metrics every 50 batches
        if batch_idx % 50 == 0:
            wandb.log({
                "phase1/batch_loss": loss,
                "phase1/batch_accuracy": acc,
                "batch": epoch * len(train_ds) + batch_idx
            })

    # Calculate epoch averages
    avg_train_loss = train_loss / train_batches
    avg_train_acc = train_acc / train_batches

    # Collect predictions for detailed metrics
    y_true, y_pred, y_pred_probs = collect_predictions(model, val_ds)

    # Calculate all metrics
    acc, top5_acc, f1_macro, f1_weighted = calculate_all_metrics(y_true, y_pred, y_pred_probs)
    # Validation
    val_loss = 0
    val_acc = 0
    val_batches = 0

    val_loss = 0
    val_batches = 0
    for x_batch, y_batch in val_ds:
        loss, _ = model.test_on_batch(x_batch, y_batch)
        val_loss += loss
        val_batches += 1
    avg_val_loss = val_loss / val_batches

    # Log epoch metrics
    wandb.log({
        "phase1/epoch": epoch + 1,
        "phase1/train_loss": avg_train_loss,
        "phase1/train_accuracy": avg_train_acc,
        "phase1/val_loss": avg_val_loss,
        "phase1/val_accuracy": acc,
        "phase1/val_top5_accuracy": top5_acc,
        "phase1/val_f1_macro": f1_macro,
        "phase1/val_f1_weighted": f1_weighted,
        "learning_rate": 1e-3
    })

    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f}")
    print("\nEvaluation Results (Validation Set)")
    print(f"Accuracy (Top-1) : {acc:.4f}")
    print(f"Top-5 Accuracy  : {top5_acc:.4f}")
    print(f"F1-score macro  : {f1_macro:.4f}")
    print(f"F1-score weight : {f1_weighted:.4f}")

# 5. PHASE 2: Unfreeze top ViT layers
for layer in backbone.layers[-4:]:
    layer.trainable = True

# 6. Re-compile with small LR
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# 7. Fine-tune with wandb logging
print("\nPhase 2: Fine-tuning top layers")
for epoch in range(5):
    print(f"\nEpoch {epoch+1}/5 (Phase 2)")

    # Training
    train_loss = 0
    train_acc = 0
    train_batches = 0

    for batch_idx, (x_batch, y_batch) in enumerate(train_ds):
        # Train on batch
        loss, acc = model.train_on_batch(x_batch, y_batch)
        train_loss += loss
        train_acc += acc
        train_batches += 1

        # Log batch metrics every 50 batches
        if batch_idx % 50 == 0:
            wandb.log({
                "phase2/batch_loss": loss,
                "phase2/batch_accuracy": acc,
                "batch": (epoch + 5) * len(train_ds) + batch_idx,
                "phase2_epoch": epoch + 1
            })

    # Calculate epoch averages
    avg_train_loss = train_loss / train_batches
    avg_train_acc = train_acc / train_batches

    # Collect predictions for detailed metrics
    y_true, y_pred, y_pred_probs = collect_predictions(model, val_ds)

    # Calculate all metrics
    acc, top5_acc, f1_macro, f1_weighted = calculate_all_metrics(y_true, y_pred, y_pred_probs)

    # Calculate validation loss
    val_loss = 0
    val_batches = 0
    for x_batch, y_batch in val_ds:
        loss, _ = model.test_on_batch(x_batch, y_batch)
        val_loss += loss
        val_batches += 1
    avg_val_loss = val_loss / val_batches
    # Log epoch metrics
    wandb.log({
        "phase2/epoch": epoch + 1,
        "phase2/train_loss": avg_train_loss,
        "phase2/train_accuracy": avg_train_acc,
        "phase2/val_loss": avg_val_loss,
        "phase2/val_accuracy": acc,
        "phase2/val_top5_accuracy": top5_acc,
        "phase2/val_f1_macro": f1_macro,
        "phase2/val_f1_weighted": f1_weighted,
        "learning_rate": 1e-5
    })

    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f}")
    print("\nEvaluation Results (Validation Set)")
    print(f"Accuracy (Top-1) : {acc:.4f}")
    print(f"Top-5 Accuracy  : {top5_acc:.4f}")
    print(f"F1-score macro  : {f1_macro:.4f}")
    print(f"F1-score weight : {f1_weighted:.4f}")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: blud (blud-fpt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vi_t_backbone (ViTBackbone)     │ (None, 197, 768)       │    85,798,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 768)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 102)            │        52,326 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 86,244,710 (329.00 MB)

 Trainable params: 446,054 (1.70 MB)

 Non-trainable params: 85,798,656 (327.30 MB)

Phase 1: Training head only (backbone frozen)

Epoch 1/5
Train Loss: 1.2875, Train Acc: 0.7433
Val Loss: 0.4641

Evaluation Results (Validation Set)
Accuracy (Top-1) : 0.9902
Top-5 Accuracy  : 0.9988
F1-score macro  : 0.9841
F1-score weight : 0.9902

Epoch 2/5
Train Loss: 0.3269, Train Acc: 0.9329
Val Loss: 0.2463

Evaluation Results (Validation Set)
Accuracy (Top-1) : 0.9927
Top-5 Accuracy  : 0.9988
F1-score macro  : 0.9885
F1-score weight : 0.9927

Epoch 3/5
Train Loss: 0.2046, Train Acc: 0.9579
Val Loss: 0.1737

Evaluation Results (Validation Set)
Accuracy (Top-1) : 0.9914
Top-5 Accuracy  : 0.9976
F1-score macro  : 0.9909
F1-score weight : 0.9914

Epoch 4/5
Train Loss: 0.1535, Train Acc: 0.9680
Val Loss: 0.1386

Evaluation Results (Validation Set)
Accuracy (Top-1) : 0.9804
Top-5 Accuracy  : 0.9976
F1-score macro  : 0.9766
F1-score weight : 0.9768

Epoch 5/5
Train Loss: 0.1285, Train Acc: 0.9731
Val Loss: 0.1210

Evaluation Results (Validation Set)
Accuracy (Top-1) : 0.9939
Top-5 Acc

In [ ]:
model.save("vit_flowers102_finetuned.keras")


In [ ]:
import tensorflow as tf
import glob

IMG_SIZE = 224
BATCH_SIZE = 32

image_paths = sorted(
    glob.glob("/kaggle/input/pytorch-challange-flower-dataset/dataset/test/*.jpg")
)

def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img

test_ds = tf.data.Dataset.from_tensor_slices(image_paths)
test_ds = test_ds.map(load_image).batch(BATCH_SIZE)



In [ ]:
import keras

model = keras.models.load_model("vit_flowers102_finetuned.keras")


In [ ]:
preds = model.predict(test_ds)


26/26 ━━━━━━━━━━━━━━━━━━━━ 20s 543ms/step


In [ ]:
# Final evaluation on validation set
print("\n" + "="*50)
print("FINAL EVALUATION ON TEST SET")
print("="*50)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/input/pytorch-challange-flower-dataset/dataset/valid",
    image_size=(224, 224),
    batch_size=32,
    shuffle=False
)

test_ds = test_ds.map(lambda x, y: (x / 255.0, y))

# Collect predictions for test set
y_true, y_pred, y_pred_probs = collect_predictions(model, test_ds)

# Calculate all metrics
acc, top5_acc, f1_macro, f1_weighted = calculate_all_metrics(y_true, y_pred, y_pred_probs)

# Calculate test loss
test_loss = 0
test_batches = 0
for x_batch, y_batch in test_ds:
    loss, _ = model.test_on_batch(x_batch, y_batch)
    test_loss += loss
    test_batches += 1
avg_test_loss = test_loss / test_batches

# Log final test metrics to wandb
wandb.log({
    "final_test_loss": avg_test_loss,
    "final_test_accuracy": acc,
    "final_test_top5_accuracy": top5_acc,
    "final_test_f1_macro": f1_macro,
    "final_test_f1_weighted": f1_weighted
})

# Print final evaluation results
print("\nEvaluation Results (Final Test Set)")
print(f"Accuracy (Top-1) : {acc:.4f}")
print(f"Top-5 Accuracy  : {top5_acc:.4f}")
print(f"F1-score macro  : {f1_macro:.4f}")
print(f"F1-score weight : {f1_weighted:.4f}")
print(f"Test Loss: {avg_test_loss:.4f}")

# Finish wandb run
wandb.finish()


FINAL EVALUATION ON TEST SET
Found 818 files belonging to 102 classes.

Evaluation Results (Final Test Set)
Accuracy (Top-1) : 0.9976
Top-5 Accuracy  : 0.9988
F1-score macro  : 0.9976
F1-score weight : 0.9975
Test Loss: 0.0057


batch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█
final_test_accuracy,▁
final_test_f1_macro,▁
final_test_f1_weighted,▁
final_test_loss,▁
final_test_top5_accuracy,▁
learning_rate,█████▁▁▁▁▁
phase1/batch_accuracy,▁▅▆▇▇▇▇▇████████████████████████████████
phase1/batch_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
phase1/epoch,▁▃▅▆█
+18,...
